This notebook explores the possibility of counting people on the photos from ZB. Here we used a model for detecting people Detectron2 by facebook.

Importing dependencies.

In [1]:
import os
import pandas as pd
import cv2
import torch
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

Paths for used folders.

In [ ]:
# === CONFIGURE ===
input_dir = "Data"       # Folder with WW2 images
output_dir = "detectron2-cascade_mask_rcnn_R_50_FPN_3x"   # Optional output folder
os.makedirs(output_dir, exist_ok=True)

Detectron2 configuration.

In [3]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
    "Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"
))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml"
)
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# Load the CSV
df = pd.read_csv('people_counts.csv', sep=';')

# Filter for counts <= 4
# filtered_df = df[df['people_count'] <= 20]

# Create list of .jpg filenames
jpg_filenames = [f"{str(id_)}.jpg" for id_ in df['id']]

In [5]:
predictor = DefaultPredictor(cfg)
model_output = []

# === PROCESS IMAGES ===
for filename in jpg_filenames:
    if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    img_path = os.path.join(input_dir, filename)
    image = cv2.imread(img_path)
    if image is None:
        print(f"Could not read {filename}")
        continue

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    outputs = predictor(image)
    instances = outputs["instances"]
    person_instances = instances[instances.pred_classes == 0]
    count = len(person_instances)

    model_output.append({
        "id": filename.removesuffix(".jpg"),
        "people_count": count
    })

    # --- Visualize and Save ---
    v = Visualizer(image_rgb, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.0)
    out = v.draw_instance_predictions(person_instances.to("cpu"))
    result_img = out.get_image()
    save_path = os.path.join(output_dir, filename)
    cv2.imwrite(save_path, cv2.cvtColor(result_img, cv2.COLOR_RGB2BGR))

# === SAVE RESULTS ===
output_df = pd.DataFrame(model_output)
output_csv_path = os.path.join(output_dir, "detectron2_output.csv")
output_df.to_csv(output_csv_path, index=False)



model_final_480dd8.pkl: 288MB [00:47, 6.10MB/s]                              
/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392020201/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
